In [4]:
# pip install openai-whisper sounddevice deep-translator gTTS pydub numpy soundfile

import whisper
import sounddevice as sd
import soundfile as sf
from deep_translator import GoogleTranslator
from gtts import gTTS
from pydub import AudioSegment
from pydub.playback import play
import tempfile
import queue
import threading
import numpy as np
import os

# Load Whisper
print("Loading Whisper model (small)...")
model = whisper.load_model("small")

fs = 16000
chunk_duration = 3
blocksize = int(fs * chunk_duration)
audio_queue = queue.Queue()

def audio_callback(indata, frames, time, status):
    if status:
        print("Audio status:", status)
    audio_queue.put(indata.copy())

def process_audio():
    while True:
        audio_chunk = audio_queue.get()
        if audio_chunk is None:
            break

        # Save chunk as WAV (closed file so pydub can read)
        temp_wav_path = f"temp_audio_{np.random.randint(0,1e6)}.wav"
        sf.write(temp_wav_path, audio_chunk, fs, subtype='PCM_16')

        # Transcribe
        result = model.transcribe(temp_wav_path, language='en')
        english_text = result["text"].strip()
        if english_text:
            print("English:", english_text)

            # Translate to Telugu
            telugu_text = GoogleTranslator(source='en', target='te').translate(english_text)
            print("Telugu:", telugu_text)

            # gTTS → MP3 (closed file)
            temp_mp3_path = f"temp_audio_{np.random.randint(0,1e6)}.mp3"
            tts = gTTS(telugu_text, lang='te')
            tts.save(temp_mp3_path)

            # Play Telugu audio via pydub
            sound = AudioSegment.from_file(temp_mp3_path, format="mp3")
            play(sound)

            # Cleanup
            os.remove(temp_wav_path)
            os.remove(temp_mp3_path)

# Start processing thread
thread = threading.Thread(target=process_audio, daemon=True)
thread.start()

# Start real-time microphone stream
try:
    print("Listening in real-time... Speak English and hear Telugu!")
    with sd.InputStream(samplerate=fs, channels=1, dtype='int16',
                        callback=audio_callback, blocksize=blocksize):
        import time
        while True:
            time.sleep(0.1)
except KeyboardInterrupt:
    print("Stopping...")
finally:
    audio_queue.put(None)
    thread.join()


ModuleNotFoundError: No module named 'pyaudioop'